# Analyze Fine-Tuned EfficientPhys Models

## Load the Predictions

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_path = utils.file_path('outputs', 'signals')
predictions_path = utils.join_paths(signals_path, 'fine_tuned_predictions.csv')

prediction = pd.read_csv(predictions_path)
prediction['signal'] = prediction['signal'].apply(eval).apply(np.array)

In [ ]:
prediction.head()

## Get the Ground Truth RR Signal

In [ ]:
ground_truth_file = utils.join_paths(signals_path, 'respiration_gt.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)
ground_truth.head()

## Analyze the Performance of the Model

In [ ]:
models = prediction['model'].unique()
models

In [ ]:
import os
import tqdm.auto as tqdm
import respiration.analysis as analysis

analysis_dir = utils.dir_path('outputs', 'analysis', mkdir=True)
analysis_file = os.path.join(analysis_dir, 'fine_tuned_analysis.csv')

if os.path.exists(analysis_file):
    analysis_results = pd.read_csv(analysis_file)
else:
    analysis_results = []

    for model_id, manifest in tqdm.tqdm(models.items()):
        for (subject, setting) in manifest['testing_scenarios']:
            scenario = prediction[(prediction['model'] == model_id) &
                                  (prediction['subject'] == subject) &
                                  (prediction['setting'] == setting)].iloc[0]
            prediction_signal = scenario['signal']

            scenario_gt = ground_truth[(ground_truth['subject'] == subject) &
                                       (ground_truth['setting'] == setting)].iloc[0]

            # Cut the signal to the same length
            gt_signal = scenario_gt['signal'][:len(prediction_signal)]

            compare = analysis.SignalCompare(
                prediction_signal,
                gt_signal,
                scenario['sampling_rate'],
            )

            for distance_name, distance_value in compare.compare_all().items():
                analysis_results.append({
                    'model_id': model_id,
                    'subject': subject,
                    'setting': setting,
                    'metric': distance_name,
                    'result': distance_value,
                })

    analysis_results = pd.DataFrame(analysis_results)
    analysis_results.to_csv(analysis_file, index=False)

In [ ]:
analysis_results.head()

In [ ]:
import matplotlib.pyplot as plt

metrics = [
    'pk_error',
    'cp_error',
    'nfcp_error',
    'psd_error',
    'distance_mse',
    'distance_pearson',
    'distance_dtw',
]
metrics.sort()

models = analysis_results['model_id'].unique()

model_scores = []

for idx, metric in enumerate(metrics):
    data = analysis_results[analysis_results['metric'] == metric]

    for model_id in models:
        model_data = data[data['model_id'] == model_id]
        mean = model_data['result'].mean()
        std = model_data['result'].std()

        model_scores.append({
            'model_id': model_id,
            'metric': metric,
            'mean': mean,
            'std': std,
        })

model_scores = pd.DataFrame(model_scores)
model_scores.to_csv(utils.join_paths(analysis_dir, 'fine_tuned_scores.csv'), index=False)
model_scores.head()

In [ ]:
fig, axes = plt.subplots(len(metrics), 1, figsize=(20, 20))
fig.tight_layout(pad=5.0)
fig.suptitle('Model Error Comparison')

for idx, metric in enumerate(metrics):
    data = model_scores[model_scores['metric'] == metric]

    axes[idx].bar(data['model_id'], data['mean'], yerr=data['std'], capsize=5)
    axes[idx].set_title(metric)
    axes[idx].set_ylabel(metric)
    axes[idx].set_xlabel('Model ID')
    axes[idx].grid()

# Store the plot as svg
figure_dir = utils.dir_path('outputs', 'figures', 'fine_tuned', mkdir=True)
utils.savefig(fig, figure_dir, 'model_error_comparison')